<a href="https://colab.research.google.com/github/zanevc5555/physics-applets/blob/main/SpinEnsembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pauli Spin Matrices

#Cant be bothered re-typing this section

**bold**
# Title
## Subtitle
<h3>Header



## Spinors

#### Eigenspinors

Eigenvectors of the Paul spin matrices, e.g,

$\sigma_z \bar{\phi_z} = \lambda \bar{\phi_z} = \pm \bar{\phi_z}$

$\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}
\begin{bmatrix} 1 \\ 0 \end{bmatrix} = \left | +_z \right \rangle$

$\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}
\begin{bmatrix}  0 \\ 1 \end{bmatrix} = -\left | -_z \right \rangle$


**In general**, spinor is a two-level quantum state corresponding to spin orientation and is the eigenvector of the spin operator $S_n = \frac{\hbar}{2}\sigma_n$.

Form of general spinor: $[\psi(\bold{x})] = \begin{bmatrix} \phi_1 \\ \phi_2 \end{bmatrix} = \phi_1 \left | +_z \right \rangle + \phi_2 \left | -_z \right \rangle$